<a href="https://colab.research.google.com/github/Santosh-Gupta/SparseTorch/blob/master/NumpyMemmapAndPytorchWeightSetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will demonstrate how to switch values between a memory mapped numpy file and a pytorch model's weights

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy
import random 
import hashlib
import os

import torch.optim as optim
from tqdm import tqdm
import sys
import tensorflow as tf
import collections
import numpy as np

from collections import deque
numpy.random.seed(12345)

In [0]:
#Define model architecture 

#Skip gram model architecture 
#Main thing to know is that it has two embedding matrixes, one for the context embedding
#   another for the target embeddings 
class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        # self.v_embeddings.weight.data.uniform_(-initrange, initrange)

    def forward(self, pos_u, pos_v, neg_v):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        score = torch.mul(emb_u, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(score)
        neg_emb_v = self.v_embeddings(neg_v)
        neg_score = torch.bmm(neg_emb_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)
        return -1 * (torch.sum(score)+torch.sum(neg_score))

    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.
        As this class only record word id, so the map from id to word has to be transfered from outside.
        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


In [0]:
#create instance of model with 20 embeddings, and embedding dimension of 7 

skip_gram_model = SkipGramModel( 20, 7)

In [0]:
skip_gram_model.parameters()

<generator object Module.parameters at 0x7f4c7cbbaca8>

In [0]:
#In this demonstration we'll only be looking at the u_embeddings embeddings 

print(skip_gram_model.u_embeddings)

Embedding(20, 7, sparse=True)


In [0]:
#Look at all the weights 

print(skip_gram_model.u_embeddings.weight)

Parameter containing:
tensor([[ 0.0578,  0.0374, -0.0301, -0.0211,  0.0535, -0.0149,  0.0655],
        [ 0.0596, -0.0679,  0.0167, -0.0334, -0.0076, -0.0709, -0.0613],
        [ 0.0532,  0.0502,  0.0086, -0.0567,  0.0010, -0.0557, -0.0175],
        [-0.0375, -0.0149, -0.0670,  0.0542, -0.0169,  0.0195,  0.0521],
        [-0.0495,  0.0456, -0.0076,  0.0372,  0.0388,  0.0066, -0.0177],
        [ 0.0657, -0.0443, -0.0228,  0.0278,  0.0139,  0.0519, -0.0108],
        [ 0.0188,  0.0707,  0.0588, -0.0697, -0.0407, -0.0531,  0.0627],
        [ 0.0064,  0.0477, -0.0046,  0.0413,  0.0040,  0.0397,  0.0645],
        [-0.0233, -0.0188, -0.0020, -0.0410,  0.0633,  0.0657,  0.0566],
        [ 0.0138, -0.0520,  0.0215, -0.0156,  0.0462,  0.0063,  0.0059],
        [ 0.0074,  0.0436,  0.0696,  0.0507,  0.0141,  0.0652, -0.0039],
        [ 0.0565, -0.0014, -0.0394, -0.0315, -0.0079,  0.0554, -0.0372],
        [-0.0019,  0.0710,  0.0073, -0.0193,  0.0506, -0.0224,  0.0446],
        [-0.0330,  0.0274,  0

In [0]:
# Look at the Pytorch tensor of the weights 

print(skip_gram_model.u_embeddings.weight.data)

tensor([[ 0.0578,  0.0374, -0.0301, -0.0211,  0.0535, -0.0149,  0.0655],
        [ 0.0596, -0.0679,  0.0167, -0.0334, -0.0076, -0.0709, -0.0613],
        [ 0.0532,  0.0502,  0.0086, -0.0567,  0.0010, -0.0557, -0.0175],
        [-0.0375, -0.0149, -0.0670,  0.0542, -0.0169,  0.0195,  0.0521],
        [-0.0495,  0.0456, -0.0076,  0.0372,  0.0388,  0.0066, -0.0177],
        [ 0.0657, -0.0443, -0.0228,  0.0278,  0.0139,  0.0519, -0.0108],
        [ 0.0188,  0.0707,  0.0588, -0.0697, -0.0407, -0.0531,  0.0627],
        [ 0.0064,  0.0477, -0.0046,  0.0413,  0.0040,  0.0397,  0.0645],
        [-0.0233, -0.0188, -0.0020, -0.0410,  0.0633,  0.0657,  0.0566],
        [ 0.0138, -0.0520,  0.0215, -0.0156,  0.0462,  0.0063,  0.0059],
        [ 0.0074,  0.0436,  0.0696,  0.0507,  0.0141,  0.0652, -0.0039],
        [ 0.0565, -0.0014, -0.0394, -0.0315, -0.0079,  0.0554, -0.0372],
        [-0.0019,  0.0710,  0.0073, -0.0193,  0.0506, -0.0224,  0.0446],
        [-0.0330,  0.0274,  0.0635,  0.0165, -0.006

In [0]:
#Look at only one embedding 

print(skip_gram_model.u_embeddings.weight.data[16])

tensor([ 0.0149,  0.0573,  0.0176,  0.0098, -0.0486,  0.0448,  0.0070])


In [0]:
#Advice from https://stackoverflow.com/questions/49768306/pytorch-tensor-to-numpy-array
# convert Pytorch tensor to numpy 

numpyArrayFromPytorch = skip_gram_model.u_embeddings.weight.data[16].cpu().detach().numpy()

In [0]:
#Numpy variable version of embedding 16

print(numpyArrayFromPytorch)

[ 0.01489079  0.05732322  0.01762484  0.00976089 -0.04861376  0.04481397
  0.00704634]


# Create Memory mapped numpy file 

In [0]:
# 20 embeddings of embedding size 7 

varMapped = np.memmap('test.mymemmap', dtype='float32', mode='w+', shape=(20 , 7))

In [0]:
#check embedding 16

varMapped[16]

memmap([0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
# set memory mapped variable from the pytorch model variable 

varMapped[16] = numpyArrayFromPytorch

In [0]:
#  We have succesfully stored the Pytorch embedding  into the memory mapped file! 

varMapped[16]

memmap([ 0.01489079,  0.05732322,  0.01762484,  0.00976089, -0.04861376,
         0.04481397,  0.00704634], dtype=float32)

# Now lets store an embedding from the memory mapped file into the Pytorch model

In [0]:
#Create random embedding to store into memory mapped file 

sampl = np.random.uniform(low=-1.0, high=1.0, size=(7,)).astype('f')
print(sampl) 

[ 0.8592322  -0.3672489  -0.6321624  -0.59087944  0.13545007  0.1910894
  0.92902905]


In [0]:
varMapped[9] 

memmap([0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
#set memory mappened embedding to random embedding

varMapped[9] = sampl

In [0]:
varMapped[9] 

memmap([ 0.8592322 , -0.3672489 , -0.6321624 , -0.59087944,  0.13545007,
         0.1910894 ,  0.92902905], dtype=float32)

In [0]:
#Set Pytorch model embedding from the memory mapped file 
#Set up from https://discuss.pytorch.org/t/layer-weight-vs-weight-data/24271/2?u=santosh-gupta
# need to temporary turn of gradient operations 

with torch.no_grad():
    skip_gram_model.u_embeddings.weight.data[9] = torch.from_numpy( varMapped[9] )

In [0]:
# Successfully set Pytorch model embedding from memory mapped file! 

skip_gram_model.u_embeddings.weight.data[9]

tensor([ 0.8592, -0.3672, -0.6322, -0.5909,  0.1355,  0.1911,  0.9290])